In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.chat_models import ChatTongyi

In [3]:
chat = ChatTongyi(temperature=0.01, top_p=0.01)

### 1. 工具的定义
- 查询当前日期的工具
- 科学计算的工具
- 实时业务信息的查询

In [4]:
# chat.invoke(input="今年是哪一年？")

In [5]:
# chat.invoke(input="0.2的平方根是多少？")

In [6]:
from langchain.tools import tool

In [7]:
@tool
def get_current_datetime() -> str:
    """
        当需要查询现在的日期或时间时，请使用此工具!
    """
    import datetime
    now = datetime.datetime.now()
    formatted_date = now.strftime("%Y-%m-%d %H:%M:%S")
    return formatted_date

In [52]:
@tool
def get_vegetable_price(item:str) -> str:
    """
        当查询 **蔬菜** 价格的时候，请调用这个工具！
        注意：
            1，如果查询的商品不属于蔬菜，请不要使用调用这个工具！
            2，返回的价格是按千克计算的，每千克等于两斤！
        
        
        入参：item为蔬菜的名称，只能是["胡萝卜", "白菜", "其它蔬菜"]中的一种！
              注意：除了 "胡萝卜" 和 "白菜" 之外的蔬菜，都属于 "其它蔬菜"！
        返参：返回该种蔬菜每千克(1千克等于2斤)的价格，用浮点数形式表达
    """
    if item == "胡萝卜":
        return "12.3元/千克"
    elif item == "白菜":
        return "8.8元/千克"
    elif item == "其它蔬菜":
        return "3.3元/千克"
    else:
        raise ValueError('入参有误，参数只能是["胡萝卜", "白菜", "其它蔬菜"]中的一种！注意：除了 "胡萝卜" 和 "白菜" 之外的蔬菜，都属于 "其它蔬菜"！')

In [56]:
@tool
def calculator(expression: str) -> str:
    """
    当涉及到数学计算时，请调用此工具！！

    注意：
        本工具使用 numexpr 库来实现计算！
        入参表达式必须是单行的 数学表达式！不支持其它格式！

    举例:
        "37593 * 67" for "37593 times 67"
        "37593**(1/5)" for "37593^(1/5)"
    """
    import math
    import numexpr
    local_dict = {"pi": math.pi, "e": math.e}
    return str(
        numexpr.evaluate(
            expression.strip(),
            global_dict={},  # restrict access to globals
            local_dict=local_dict,  # add common mathematical functions
        )
    )

In [57]:
# 工具集
tools = [get_current_datetime, get_vegetable_price, calculator]

In [58]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage

In [59]:
# 创建一个 代理 agent
agent = create_react_agent(model=chat, tools=tools)

In [60]:
# chat.invoke(input="今年是哪一年？")

In [61]:
# agent.invoke(input={"messages": [HumanMessage(content="今年是哪一年？")]})

In [62]:
# results = agent.invoke(input={"messages": [HumanMessage(content="我有一个2000年的女朋友，她比我大3岁，我的年龄的平方是多少？")]})

In [63]:
# chat.invoke(input="土豆多少钱一斤？")

In [64]:
agent.invoke(input={"messages": [HumanMessage(content="0.25斤白菜多少钱？")]})

{'messages': [HumanMessage(content='0.25斤白菜多少钱？', additional_kwargs={}, response_metadata={}, id='8fc17033-f4af-4f46-bbeb-ae0bc1ee54d2'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_vegetable_price', 'arguments': '{"item": "白菜"}'}, 'index': 0, 'id': 'call_b696c6fc018b4fcf93b51b', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': 'f90d1ff0-f76d-9907-a2d1-3c15eeedafe8', 'token_usage': {'input_tokens': 488, 'output_tokens': 20, 'total_tokens': 508}}, id='run-3c27a0b7-f375-4038-8e18-8527f529e908-0', tool_calls=[{'name': 'get_vegetable_price', 'args': {'item': '白菜'}, 'id': 'call_b696c6fc018b4fcf93b51b', 'type': 'tool_call'}]),
  ToolMessage(content='8.8元/千克', name='get_vegetable_price', id='55d4a22e-f591-4a12-b289-d05f1e91a2ce', tool_call_id='call_b696c6fc018b4fcf93b51b'),
  AIMessage(content='白菜的价格是8.8元每千克，因此0.25斤（即0.125千克）白菜的价格是8.8元 * 0.125 = 1.1元。', additional_kwargs={}, response_met

In [65]:
agent.invoke(input={"messages": [HumanMessage(content="皮鞋多少钱一斤？")]})

{'messages': [HumanMessage(content='皮鞋多少钱一斤？', additional_kwargs={}, response_metadata={}, id='05d84519-f39f-43ee-8e07-b3c8cfa6be34'),
  AIMessage(content='这个问题涉及到的商品并不是蔬菜，因此无法通过`get_vegetable_price`工具查询。皮鞋的价格会根据品牌、材质、制作工艺等多种因素变化，通常不会以“斤”作为计量单位。建议您在具体的商店或者电商平台查找对应款式和品牌的皮鞋价格。如果您是在寻找某种特定类型的皮鞋（比如某种材质的皮鞋），请提供更详细的信息，这样我可以为您提供更准确的指导。', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': 'c145365d-a4b8-9fff-96cd-e9b97fe12510', 'token_usage': {'input_tokens': 486, 'output_tokens': 90, 'total_tokens': 576}}, id='run-8ad52bae-4fcf-49a2-862b-0c85839dfb11-0')]}

In [70]:
results = agent.invoke(input={"messages": [HumanMessage(content="假如新中国成立多少年我就买多少斤胡萝卜，那么我得准备多少钱呢？")]})

In [71]:
results

{'messages': [HumanMessage(content='假如新中国成立多少年我就买多少斤胡萝卜，那么我得准备多少钱呢？', additional_kwargs={}, response_metadata={}, id='80afb7de-eedf-4da9-b18f-fa81d051922b'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_vegetable_price', 'arguments': '{"item": "胡萝卜"}'}, 'index': 0, 'id': 'call_fb98157df7c44d5c92f3bb', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '8e1a1d82-8353-9afc-a43b-49c1aff28b44', 'token_usage': {'input_tokens': 498, 'output_tokens': 20, 'total_tokens': 518}}, id='run-525793a6-5a35-4d22-b1e6-6b131bf3ef1b-0', tool_calls=[{'name': 'get_vegetable_price', 'args': {'item': '胡萝卜'}, 'id': 'call_fb98157df7c44d5c92f3bb', 'type': 'tool_call'}]),
  ToolMessage(content='12.3元/千克', name='get_vegetable_price', id='e5aad34c-e8a9-4bab-a1a2-ce0460feae89', tool_call_id='call_fb98157df7c44d5c92f3bb'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_current